In [ ]:
'''
Goal:Check hurskainen vessel_size_gradient
'''

In [ ]:
import scanpy as sc
import scanpy.external as sce
import os 
import pandas as pd 
import numpy as np
import seaborn as sns
from functions import compare_obs_values_within_groups_to_excel
import matplotlib.pyplot as plt
import palantir
# from statannotations.Annotator import Annotator

adata_name='venous_ec'
figures = "data/figures/figures/hurskainen"
data = "data/single_cell_files/scanpy_files"

os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15

In [ ]:
endo_adata= sc.read(f'{figures}/vessel_size.gz.h5ad')
sc.pl.umap(endo_adata,color=['Cell Subtype'],cmap='viridis',size=size,legend_fontsize=10, legend_fontoutline=1,frameon=False,save='hurskainencellsubtype.png')


In [ ]:
source = '/home/carsten/alvira_bioinformatics/data/external_datasets/hurskainen2021_hyperoxia_lung/raw'
metadata = pd.read_csv(os.path.join(source, "GSE151974_cell_metadata_postfilter.csv.gz"),
                       compression='gzip', 
                       header=0,
                       index_col= 0,
                       sep=',', 
                      )

adata = sc.read_csv(os.path.join(source,"GSE151974_raw_umi_matrix_postfilter.csv.gz"),
                       first_column_names=True, 
                       delimiter=',', 
                    ).T
adata.obs = metadata
adata.obs.rename(columns = {'Age':'Timepoint',
                  'Oxygen': 'Treatment',
                  'CellType': 'celltype'},
                 inplace = True)

adata.obs['celltype'].replace({'Cap':'Cap1','Cap-a':'Cap2'},inplace=True)
adata.obs['timepoint_treatment'] =adata.obs['Timepoint'].astype(str) + '_' + adata.obs['Treatment'].astype(str)
sc.pp.calculate_qc_metrics(
        adata,
        expr_type="counts",
        log1p=True,
        inplace=True,
    )
## read in cell cycle genes and score
cell_cycle_genes = [x.strip() for x in open('/home/carsten/alvira_bioinformatics/venous_ec_scrnaseq/data/outside_data/regev_lab_cell_cycle_genes.txt')]
cell_cycle_genes = [x.lower().capitalize() for x in cell_cycle_genes]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
sc.pp.normalize_total(adata, key_added=None, target_sum=1e4)
sc.pp.log1p(adata)
sc.tl.score_genes(adata, ['Mki67', 'Top2a', 'Birc5', 'Hmgb2', 'Cenpf'], score_name='proliferation_score')
adata.layers["log1p"] = adata.X.copy()


In [ ]:
endo_cts = ['Art','Cap1',
'Vein',
]
endo_adata = adata[adata.obs['celltype'].isin(endo_cts)].copy()
sc.tl.score_genes_cell_cycle(endo_adata, s_genes=s_genes, g2m_genes=g2m_genes)
sc.pp.regress_out(endo_adata, ['S_score', 'G2M_score'])
endo_adata.layers['cc_regress'] = endo_adata.X.copy()

In [ ]:

sc.pp.highly_variable_genes(endo_adata,batch_key='orig.ident')
sc.pp.pca(endo_adata, mask_var="highly_variable")
sce.pp.harmony_integrate(endo_adata,'orig.ident',max_iter_harmony = 20)
sc.pp.neighbors(endo_adata, use_rep="X_pca_harmony")
sc.tl.leiden(endo_adata, key_added="leiden")
endo_adata.obs['Cell Subtype'] = endo_adata.obs['leiden'].map({'0':'Cap1','1':'Cap1','2':'Cap1','3':'Cap1','4':'Cap1','5':'Arterial EC','6':'Venous EC','7':'Cap1',})
endo_adata.uns['Cell Subtype_colors']= ['#4A90E2','#9B59B6','#E35D6A']

sc.tl.umap(endo_adata,min_dist=0.5)
sc.tl.rank_genes_groups(endo_adata,'leiden',method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(endo_adata,dendrogram=False)

for color in ['celltype','Timepoint','Treatment','leiden','Gja5','Slc6a2','Kit','Hey1','Nr2f2','Mecom','Eln','Mgp','Col4a1','Col4a2']:
    sc.pl.umap(endo_adata,color=color,use_raw=False)

In [ ]:
endo_adata.X = endo_adata.layers['log1p'].copy()

In [ ]:
sc.pl.umap(endo_adata,color='Cell Subtype')

In [ ]:
import palantir
import cellrank as cr
import scvelo as scv

root_ct = 'Cap1'
terminal_cts = ['Arterial EC','Venous EC']
celltype='Cell Subtype'

palantir.utils.run_diffusion_maps(endo_adata,
                                           n_components=5)
fig = palantir.plot.plot_diffusion_components(endo_adata)[0]
fig.tight_layout()
fig.savefig(f'{figures}/hurskainenpalantir_diffusion_components.png')
plt.close()
palantir.utils.determine_multiscale_space(endo_adata)

palantir.utils.run_magic_imputation(endo_adata)
subset = endo_adata[endo_adata.obs[celltype] == root_ct]
umap1_values = subset.obsm['X_umap'][:, 1]
min_idx = np.argmin(umap1_values)
root_cell = subset.obs_names[min_idx]
terminal_states = []
for ct in terminal_cts:
    subset = endo_adata[endo_adata.obs[celltype] == ct]
    if ct =='Arterial EC':
        # Get the index (obs_names) of the cell with the min UMAP1 (usually component 0)
        umap1_values = subset.obsm['X_umap'][:, 1]
        max_idx = np.argmax(umap1_values)
        # Return the cell name
        terminal_states.append(subset.obs_names[max_idx])
    else:
        umap1_values = subset.obsm['X_umap'][:, 0]
        max_idx = np.argmax(umap1_values)
        # Return the cell name
        terminal_states.append(subset.obs_names[max_idx])
        
terminal_states = pd.Series(index=terminal_states, data=terminal_cts, dtype='object')

fig = palantir.plot.highlight_cells_on_umap(endo_adata, [root_cell]+terminal_states)[0]
fig.tight_layout()
fig.savefig(f'{figures}/hurskainenpalantir_terminal_cells.png')
plt.close()

palantir.core.run_palantir(
    endo_adata, root_cell, num_waypoints=500, terminal_states=terminal_states
)

fig = palantir.plot.plot_palantir_results(endo_adata, s=3)
fig.tight_layout()
fig.savefig(f'{figures}/hurskainenpalantir_results.png')
plt.close()
iroot = endo_adata.obs.index.get_loc(root_cell)
endo_adata.uns["iroot"] = iroot
sc.tl.dpt(endo_adata)

try:
    palantir.presults.select_branch_cells(endo_adata, q=.01, eps=.01,pseudo_time_key='dpt_pseudotime')

    fig = palantir.plot.plot_branch_selection(endo_adata)
    fig.tight_layout()
    fig.savefig(f'{figures}/hurskainenpalantir_branch_selection.png')
    plt.close()

except:
    pass

sc.tl.diffmap(endo_adata)
scv.pl.scatter(
    endo_adata,
    basis="diffmap",
    c=[celltype, iroot],
    legend_loc="right",
    components=["2, 3"],
    show=False,
    save=f'hurskainendiffmap_{celltype}_root_cell.png'
)


sc.pl.embedding(
    endo_adata,
    basis="umap",
    color=["dpt_pseudotime", "palantir_pseudotime"],
    color_map="viridis",
    show=False,
    save='_hurskainen_pseudotimes.png'
)

palantir.presults.compute_gene_trends(
    endo_adata,
    expression_key="MAGIC_imputed_data",
    pseudo_time_key='dpt_pseudotime'
)

pk = cr.kernels.PseudotimeKernel(endo_adata, time_key="palantir_pseudotime")
pk.compute_transition_matrix()
pk.plot_projection(basis="umap", color=celltype, recompute=True,legend_loc='right margin',
                         save=f'{figures}/hurskainenpalantir_pseudotime_stream.png')


In [ ]:
import pandas as pd
from scipy.stats import spearmanr

def correlate_genes_with_pseudotime(adata, layer=None, method='spearman',pseudotime='dpt_pseudotime'):
    """
    Correlates all genes with pseudotime in an AnnData object.

    Parameters:
    - adata: AnnData object with pseudotime in `adata.obs['pseudotime']`
    - layer: (Optional) Layer to use instead of adata.X (e.g., 'log1p', 'counts')
    - method: Correlation method, either 'spearman' (default) or 'pearson'

    Returns:
    - pandas DataFrame with genes as index and columns: ['correlation', 'pval']
    """
    if pseudotime not in adata.obs:
        raise ValueError("Pseudotime must be stored in adata.obs['pseudotime'].")

    # Get expression matrix
    X = adata.X if layer is None else adata.layers[layer]
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X.toarray() if hasattr(X, "toarray") else X,
                         index=adata.obs_names, columns=adata.var_names)

    # Extract pseudotime
    pseudotime = adata.obs[pseudotime]

    # Run correlation
    results = []
    for gene in X.columns:
        if method == 'spearman':
            corr, pval = spearmanr(X[gene], pseudotime)
        elif method == 'pearson':
            corr, pval = X[gene].corr(pseudotime), None  # Pearson p-value not computed here
        else:
            raise ValueError("Method must be 'spearman' or 'pearson'.")
        results.append((gene, corr, pval))

    result_df = pd.DataFrame(results, columns=['gene', 'correlation', 'pval']).set_index('gene')
    return result_df.sort_values('correlation', ascending=False)

In [ ]:
corr_dfs = {}
for ct in ['Arterial EC','Venous EC']:
    ct_adata = endo_adata[endo_adata.obs['Cell Subtype']==ct]
    df = correlate_genes_with_pseudotime(ct_adata,method='pearson',pseudotime='palantir_pseudotime')
    corr_dfs[ct]=df.dropna(how='all')

In [ ]:
top_n_genes=50
arterial_large_genes = corr_dfs['Arterial EC'].head(top_n_genes).index.tolist()
venous_large_genes = corr_dfs['Venous EC'].head(top_n_genes).index.tolist()
arterial_small_genes = corr_dfs['Arterial EC'].tail(top_n_genes).index.tolist()[::-1]
venous_small_genes = corr_dfs['Venous EC'].tail(top_n_genes).index.tolist()[::-1]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn3

# Define your lists


# Create the Venn diagram
venn = venn2([set(arterial_large_genes), set(venous_large_genes)], 
             set_labels=('Arterial', 'Venous'), 
             set_colors=('#4A90E2', '#E35D6A'), 
             alpha=0.7)

# Optional: Customize font size
for text in venn.set_labels:
    text.set_fontsize(12)
for text in venn.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the plot
plt.title("Top 50 genes positively correlated with pseudotime")
plt.savefig(f'{figures}/hurskainenvenn_diagram_large.png',dpi=300,bbox_inches='tight')
plt.close()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

# Define your lists


# Create the Venn diagram
venn = venn2([set(arterial_small_genes), set(venous_small_genes)], 
             set_labels=('Arterial', 'Venous'), 
             set_colors=('#4A90E2', '#E35D6A'), 
             alpha=0.7)

# Optional: Customize font size
for text in venn.set_labels:
    text.set_fontsize(12)
for text in venn.subset_labels:
    if text:
        text.set_fontsize(12)

# Show the plot
plt.title("Top 50 genes negatively correlated with pseudotime")
plt.savefig(f'{figures}/hurskainenvenn_diagram_small.png',dpi=300,bbox_inches='tight')
plt.close()

In [ ]:
large_genes = [x for x in arterial_large_genes if x in venous_large_genes]
small_genes = [x for x in arterial_small_genes if x in venous_small_genes]
sc.tl.score_genes(endo_adata,large_genes,score_name='large_score')
sc.tl.score_genes(endo_adata,small_genes,score_name='small_score')
endo_adata.obs['Vessel size score'] = endo_adata.obs['large_score'] - endo_adata.obs['small_score']


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
def normalize_dataframe(df):
    # Initialize the MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit the scaler on the data and transform each column
    df_normalized = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)
    return df_normalized
endo_adata.obs['Vessel size score'] = scaler.fit_transform(endo_adata.obs[['Vessel size score']])
endo_adata.obs['Vessel size category'] = pd.cut(endo_adata.obs['Vessel size score'], bins=4,labels=['capillary','small','medium','large'])
sc.pl.umap(endo_adata,color=['Vessel size score'],cmap='Oranges',size=size,frameon=False,save='_hurskainenvessel_size_score.png')
sc.pl.umap(endo_adata,color=['Vessel size category'],cmap='viridis',size=size,frameon=False,save='_hurskainenvessel_size_category.png')
sc.pl.umap(endo_adata,color=['Cell Subtype'],cmap='viridis',size=size,legend_loc='on data',legend_fontsize=10, legend_fontoutline=1,frameon=False,save='hurskainencellsubtype.png')
sc.pl.umap(endo_adata,color=['Mgp'],cmap='viridis',size=size,frameon=False,save='hurskainenmgp.png')
sc.pl.umap(endo_adata,color=['Col4a1'],cmap='viridis',size=size,frameon=False,save='hurskainencol4a1.png')
sc.pl.umap(endo_adata,color=['Col4a2'],cmap='viridis',size=size,frameon=False,save='hurskainencol4a2.png')
sc.pl.umap(endo_adata,color=['Eln'],cmap='viridis',size=size,frameon=False,save='hurskaineneln.png')
sc.pl.umap(endo_adata,color=['Hey1'],cmap='viridis',size=size,frameon=False,save='hurskainenhey1.png')
sc.pl.umap(endo_adata,color=['Nr2f2'],cmap='viridis',size=size,frameon=False,save='hurskainennr2f2.png')


In [ ]:
sc.pl.umap(endo_adata,color = large_genes + small_genes,cmap='viridis',hspace=0.5,save='hurskainen_allsize.png')

In [ ]:
endo_adata.write(f'{figures}/vessel_size.gz.h5ad',compression='gzip')

In [ ]:
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
import matplotlib.lines as mlines

def custom_dotplot(
    adata, genes, x_obs, y_obs,
    x_order=None, y_order=None,
    min_expr=0.1, cmap='RdBu_r',
    dot_max_size=300, pad=0.5,
    scale_by_gene=False,
    figsize=None,
    save=None, dpi=300,
    show_gridlines=False
):
    """
    Custom dotplot that mimics scanpy's style but allows:
    - Custom x/y groupings
    - Split x-axis for condition + gene
    - Color = average expression, Size = percent expressing
    - Optional scaling and exporting
    """
    adata.obs[x_obs] = adata.obs[x_obs].astype('category')
    adata.obs[y_obs] = adata.obs[y_obs].astype('category')

    if x_order is None:
        x_order = adata.obs[x_obs].cat.categories.tolist()
    if y_order is None:
        y_order = adata.obs[y_obs].cat.categories.tolist()

    df = sc.get.obs_df(adata, keys=[x_obs, y_obs] + genes, layer=None)

    results = []
    for gene in genes:
        for x_val in x_order:
            for y_val in y_order:
                group = df[(df[x_obs] == x_val) & (df[y_obs] == y_val)]
                if group.shape[0] == 0:
                    continue
                expr = group[gene]
                avg_expr = expr.mean()
                prop_expr = (expr > min_expr).mean()
                results.append({
                    "gene": gene,
                    "x_group": x_val,
                    "y_group": y_val,
                    "avg_expr": avg_expr,
                    "prop_expr": prop_expr
                })

    plot_df = pd.DataFrame(results)
    plot_df["x_label"] = plot_df["gene"] + "\n" + plot_df["x_group"]

    # Scale avg_expr within gene
    if scale_by_gene:
        plot_df["scaled_expr"] = plot_df.groupby("gene")["avg_expr"].transform(
            lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8)
        )
        color_col = "scaled_expr"
    else:
        color_col = "avg_expr"

    # X/Y axis label arrangement
    x_labels = []
    x_groups = []
    genes_list = []
    for gene in genes:
        for x_val in x_order:
            x_labels.append(f"{x_val}\n{gene}")
            x_groups.append(x_val)
            genes_list.append(gene)

    y_labels = y_order

    if figsize is None:
        figsize = (len(x_labels) * 0.6 + 2, len(y_labels) * 0.6 + 2)

    fig, ax = plt.subplots(figsize=figsize)

    # Scatter plot
    for _, row in plot_df.iterrows():
        x = x_labels.index(f"{row['x_group']}\n{row['gene']}")
        y = y_labels.index(row["y_group"])
        ax.scatter(
            x, y,
            s=row["prop_expr"] * dot_max_size,
            c=[row[color_col]],
            cmap=cmap,
            vmin=plot_df[color_col].min(),
            vmax=plot_df[color_col].max(),
            edgecolor='black',
            linewidth=0.5
        )

    # Optional gridlines
    if show_gridlines:
        for y in range(len(y_labels)):
            ax.axhline(y, color='lightgray', linestyle=':', linewidth=0.5)

    # Vertical dashed lines between gene groups
    for i in range(1, len(genes)):
        xpos = i * len(x_order) - 0.5
        ax.axvline(x=xpos, color='gray', linestyle='--', linewidth=1)

    ax.set_xticks(range(len(x_labels)))
    ax.set_xticklabels(x_groups, rotation=0, ha='center')
    ax.set_yticks(range(len(y_labels)))
    ax.set_yticklabels(y_labels)
    ax.set_xlim(-pad, len(x_labels) - 1 + pad)
    ax.set_ylim(-pad, len(y_labels) - 1 + pad)
    ax.invert_yaxis()
    ax.set_xlabel('')
    ax.set_ylabel(y_obs)

    # # Add gene names on second x-axis
    ax_gene = ax.secondary_xaxis('top')
    gene_locs = range(len(x_labels))
    gene_locs = [(gene_locs[i] + gene_locs[i + 1]) / 2 for i in range(len(gene_locs) - 1)][::2]
    ax_gene.set_xticks(gene_locs)
    # if len(set(genes_list)) == 1:
    #     ax_gene.set_xticklabels(genes_list[::2], rotation=0, ha='center')
    # else:
    ax_gene.set_xticklabels(genes_list[::2], rotation=0, ha='center')
    # # ax_gene.set_xlabel("Gene")

    # Colorbar
    norm = plt.Normalize(plot_df[color_col].min(), plot_df[color_col].max())
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, location='right', pad=0.02)
    cbar.set_label('Mean expression\nin group')

    # Dot size legend
    prop_vals = plot_df["prop_expr"]
    min_pct = int(np.floor(prop_vals.min() * 100 / 5) * 5)
    max_pct = int(np.ceil(prop_vals.max() * 100 / 5) * 5)
    possible_labels = [i for i in range(min_pct, max_pct + 1) if i % 5 == 0]
    num_labels = min(4, len(possible_labels))
    legend_labels = np.linspace(min_pct, max_pct, num_labels, dtype=int)

    handles = [
        plt.scatter([], [], s=(pct / 100) * dot_max_size, c='gray',
                    edgecolors='black', linewidth=0.5, label=f"{pct}%")
        for pct in legend_labels
    ]

    # Move axis to left to make room for both legends
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.72, box.height])

        # --- Dot size legend with clean rounding ---
    min_pct_raw = plot_df["prop_expr"].min() * 100
    max_pct_raw = plot_df["prop_expr"].max() * 100
    
    min_pct = int(np.floor(min_pct_raw / 5) * 5)
    max_pct = int(np.ceil(max_pct_raw / 5) * 5)
    
    # Generate 4 nicely rounded ticks between min and max
    if max_pct - min_pct < 15:
        ticks = np.linspace(min_pct, max_pct, 4)
    else:
        ticks = np.round(np.linspace(min_pct, max_pct, 4) / 5) * 5
    
    ticks = np.clip(ticks, 0, 100).astype(int)
    
    handles = [
        plt.scatter([], [], s=(p / 100) * dot_max_size, c='gray', edgecolors='black')
        for p in ticks
    ]
    labels = [f"{p}%" for p in ticks]
    
    fig.legend(
        handles,
        labels,
        title="Pct. Expressing",
        loc='center right',
        bbox_to_anchor=(1.40, 0.6),
        frameon=True
    )

    plt.tight_layout()

    if save:
        plt.savefig(save, dpi=dpi, bbox_inches='tight')
        print(f"Saved to {save}")
    else:
        plt.show()
adata_ven = endo_adata[endo_adata.obs['Cell Subtype']=='Venous EC']
adata_ven.obs['proliferation_score'] = normalize_dataframe(adata.obs[['proliferation_score']])
df = sc.get.obs_df(adata_ven,['proliferation_score','Vessel size category', 'Treatment','Timepoint'])
df.rename(columns={'proliferation_score':'Proliferation score'},inplace=True)
df = df.loc[df['Vessel size category']!= 'capillary']
for tp in endo_adata.obs['Timepoint'].cat.categories:
    fig, axs = plt.subplots(1, 2,figsize=(3,2),sharey=True)
    axs = axs.ravel()
    hue_order = ['small', 'medium', 'large']
    palette = endo_adata.uns['Vessel size category_colors'][1:]
    
    for i, treat in enumerate(['Normoxia', 'Hyperoxia']):
        ax = sns.kdeplot(
            data=df[(df['Treatment'] == treat)&(df['Timepoint'] == tp)],
            x="Proliferation score",
            hue='Vessel size category',
            hue_order=hue_order,
            palette=palette,
            common_norm=False,
            ax=axs[i]
        )
    
        ax.set_title(treat)
        ax.set_ylabel('Proportion')
        ax.set_xlabel('')
        ax.set_xticks([0,1])
        ax.set_xticklabels(['0', '1'])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
        if i == 0:
            # ✅ Line-style legend handles in reverse order
            handles = [
                mlines.Line2D([], [], color=palette[j], label=label, linewidth=2)
                for j, label in reversed(list(enumerate(hue_order)))
            ]
            ax.legend(handles=handles[::-1], frameon=False, fontsize="8", title='', loc="upper right")
        else:
            ax.get_legend().remove()
    
    fig.supxlabel('Proliferation score', y=0.15, x=0.52)
    fig.tight_layout()
    fig.savefig(f'{figures}/histplot_venous_ec_treat_proliferation_score_{tp}.png', dpi=300, bbox_inches='tight')
    plt.close()
    adata_ven_tp= adata_ven[adata_ven.obs['Timepoint']==tp]
    adata_ven_tp.obs['Treatment'] = adata_ven_tp.obs['Treatment'].str[0]
    custom_dotplot(adata_ven_tp[adata_ven_tp.obs['Vessel size category']!='capillary'],
                   ['Mki67', 'Top2a', 'Birc5', 'Hmgb2', 'Cenpf'],
                   scale_by_gene=True, 
                   x_obs='Treatment',
                   y_obs='Vessel size category',
                   x_order=['N','H'],
                   cmap='Reds',
                   dot_max_size=100,
                   figsize=(4,2),
                   save=f'{figures}/dotplot_proliferation_vec_{tp}.png')

In [ ]:

fig, axs = plt.subplots(1, 2,figsize=(3,2),sharey=True)
axs = axs.ravel()
hue_order = ['P3', 'P7', 'P14']
palette = endo_adata.uns['Timepoint_colors']
df = adata_ven.obs[['proliferation_score','Timepoint','Treatment']]
for i, treat in enumerate(['Normoxia', 'Hyperoxia']):
    ax = sns.kdeplot(
        data=df.loc[(df['Treatment'] == treat)],
        x="proliferation_score",
        hue='Timepoint',
        hue_order=hue_order,
        palette=palette,
        common_norm=False,
        ax=axs[i]
    )

    ax.set_title(treat)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('')
    ax.set_xticks([0,1])
    ax.set_xticklabels(['0', '1'])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    if i == 0:
        # ✅ Line-style legend handles in reverse order
        handles = [
            mlines.Line2D([], [], color=palette[j], label=label, linewidth=2)
            for j, label in reversed(list(enumerate(hue_order)))
        ]
        ax.legend(handles=handles[::-1], frameon=False, fontsize="8", title='', loc="upper right")
    else:
        ax.get_legend().remove()

fig.supxlabel('Proliferation score', y=0.15, x=0.52)
fig.tight_layout()
fig.savefig(f'{figures}/histplot_venous_ec_timepoint_proliferation.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2,2))
palantir.plot.plot_trajectory(
    endo_adata, # your anndata
    "Arterial EC", # the branch to plot
    cell_color="dpt_pseudotime", # the ad.obs colum to color the cells by
    n_arrows=5, # the number of arrow heads along the path
    color='#4A90E2', # the color of the path and arrow heads
    scanpy_kwargs=dict(cmap="viridis",size=size), # arguments passed to scanpy.pl.embedding
    arrowprops=dict(arrowstyle="->,head_length=.25,head_width=.25", lw=2), # appearance of the arrow heads
    lw=2, # thickness of the path
ax=ax
    # pseudotime_interval=(0, .9), # interval of the pseudotime to cover with the path
)
fig.tight_layout()

fig.savefig(f'{figures}/hurskainenpalantir_art_trajectory.png')
plt.close()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2,2))
palantir.plot.plot_trajectory(
    endo_adata, # your anndata
    "Venous EC", # the branch to plot
    cell_color="dpt_pseudotime", # the ad.obs colum to color the cells by
    n_arrows=5, # the number of arrow heads along the path
    color='#E35D6A', # the color of the path and arrow heads
    scanpy_kwargs=dict(cmap="viridis",size=size), # arguments passed to scanpy.pl.embedding
    arrowprops=dict(arrowstyle="->,head_length=.25,head_width=.25", lw=2), # appearance of the arrow heads
    lw=2, # thickness of the path
ax=ax
    # pseudotime_interval=(0, .9), # interval of the pseudotime to cover with the path
)
fig.tight_layout()
fig.savefig(f'{figures}/hurskainenpalantir_ven_trajectory.png')
plt.close()

In [ ]:

for ct in ['Arterial EC','Venous EC']:
    df = sc.get.obs_df(endo_adata,['Treatment','Timepoint','Cell Subtype','Vessel size score'])
    df_ct = df.loc[df['Cell Subtype'] == ct]
    for tp in ['P3','P7','P14']:
        df = df_ct.loc[df_ct['Timepoint']==tp]
        fig, ax = plt.subplots(1, figsize=(3,2),sharey=True)
        hue_order = ['Normoxia','Hyperoxia']
        # palette =['blue','red'],
    
        ax = sns.kdeplot(
            data=df,
            x="Vessel size score",
            hue='Treatment',
            hue_order=hue_order,
            cut=0,
            # palette=palette,
            common_norm=False,
            # stat='probability',
            # element='poly',
            # fill=False,
            # common_norm=False,
            # bins=20,
        )
    
        ax.set_title(f'{ct}\nVessel size distribution')
        ax.set_ylabel('Proportion')
        # ax.set_ylim([0,0.1])
        # ax.set_xlim([0.2,1])
        # ax.set_xticklabels([])
        # ax.set_xlabel('')
        # ax.get_legend().remove()
        
        # fig.supxlabel('Vessel size score\n0-1', y=0.15, x=0.52)
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        fig.tight_layout()
        fig.savefig(f'{figures}/histplot_vessel_size_disease_{ct}_{tp}.png', dpi=300, bbox_inches='tight')
        plt.close()


In [ ]:
os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15

In [ ]:
sc.pl.umap(endo_adata,color=['Vessel size score'],cmap='viridis',size=size,frameon=False)


In [ ]:
for tp in ['P3','P7','P14']:
    tp_adata = endo_adata[endo_adata.obs['Timepoint']==tp]
    tp_adata.X = tp_adata.layers['cc_regress']
    figures_tp = f"data/figures/figures/hurskainen/{tp}"
    os.makedirs(figures_tp, exist_ok=True)
    sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
    sc.settings.figdir = figures_tp
    sc.pp.highly_variable_genes(tp_adata,batch_key='orig.ident')
    sc.pp.pca(tp_adata, mask_var="highly_variable")
    sce.pp.harmony_integrate(tp_adata,'orig.ident',max_iter_harmony = 20)
    sc.pp.neighbors(tp_adata, use_rep="X_pca")
    sc.tl.leiden(tp_adata, key_added="leiden")
    sc.tl.umap(tp_adata,min_dist=1)
    tp_adata.X = tp_adata.layers['log1p']
    sc.tl.rank_genes_groups(tp_adata,'leiden',method='wilcoxon')
    sc.pl.rank_genes_groups_dotplot(tp_adata,dendrogram=False)
    for color in ['celltype','Cell Subtype','Timepoint','Treatment','leiden','Gja5','Slc6a2','Kit','Hey1','Nr2f2','Mecom','Eln','Mgp','Col4a1','Col4a2']:
        sc.pl.umap(tp_adata,color=color,use_raw=False,cmap='viridis',size=size,frameon=False,save=f'{tp}_{color}.png')
    for treat in ['Normoxia','Hyperoxia']:
        treat_adata = tp_adata[tp_adata.obs['Treatment']==treat]
        treat_adata.X = treat_adata.layers['cc_regress']
        figures_tp = f"data/figures/figures/hurskainen/{tp}/{treat}"
        os.makedirs(figures_tp, exist_ok=True)
        sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
        sc.settings.figdir = figures_tp
        sc.pp.highly_variable_genes(treat_adata,batch_key='orig.ident')
        sc.pp.pca(treat_adata, mask_var="highly_variable")
        sce.pp.harmony_integrate(treat_adata,'orig.ident',max_iter_harmony = 20)
        sc.pp.neighbors(treat_adata, use_rep="X_pca")
        sc.tl.leiden(treat_adata, key_added="leiden")
    
        sc.tl.umap(treat_adata,min_dist=1)
        treat_adata.X = treat_adata.layers['log1p']
        sc.tl.rank_genes_groups(treat_adata,'leiden',method='wilcoxon')
        sc.pl.rank_genes_groups_dotplot(treat_adata,dendrogram=False)
        for color in ['celltype','Cell Subtype','Timepoint','Treatment','leiden','Gja5','Slc6a2','Kit','Hey1','Nr2f2','Mecom','Eln','Mgp','Col4a1','Col4a2']:
            sc.pl.umap(treat_adata,color=color,use_raw=False,cmap='viridis',size=size,frameon=False,save=f'{tp}_{color}.png')

In [ ]:
for treat in ['Normoxia','Hyperoxia']:
    treat_adata = endo_adata[endo_adata.obs['Treatment']==treat]
    treat_adata.X = treat_adata.layers['cc_regress']
    figures_tp = f"data/figures/figures/hurskainen/{treat}"
    os.makedirs(figures_tp, exist_ok=True)
    sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
    sc.settings.figdir = figures_tp
    sc.pp.highly_variable_genes(treat_adata,batch_key='orig.ident')
    sc.pp.pca(treat_adata, mask_var="highly_variable")
    sce.pp.harmony_integrate(treat_adata,'orig.ident',max_iter_harmony = 20)
    sc.pp.neighbors(treat_adata, use_rep="X_pca")
    sc.tl.leiden(treat_adata, key_added="leiden")

    sc.tl.umap(treat_adata,min_dist=1)
    tp_adata.X = tp_adata.layers['log1p']
    sc.tl.rank_genes_groups(treat_adata,'leiden',method='wilcoxon')
    sc.pl.rank_genes_groups_dotplot(treat_adata,dendrogram=False)
    for color in ['celltype','Cell Subtype','Timepoint','Treatment','leiden','Gja5','Slc6a2','Kit','Hey1','Nr2f2','Mecom','Eln','Mgp','Col4a1','Col4a2']:
        sc.pl.umap(treat_adata,color=color,use_raw=False,cmap='viridis',size=size,frameon=False,save=f'{tp}_{color}.png')

In [ ]:
os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15

In [ ]:
mask = endo_adata.obs['Cell Subtype'] =='Venous EC'
sc.pl.umap(endo_adata.copy(),color=['Timepoint'],mask_obs=mask,size=size,na_in_legend=False,frameon=False,save='hurskainen_ven_timepoint.png')
sc.pl.umap(endo_adata.copy(),color=['Treatment'],palette=endo_adata.uns['Treatment_colors'][::-1],mask_obs=mask,size=size,na_in_legend=False,frameon=False,save='hurskainen_ven_treatment.png')
sc.pl.umap(endo_adata.copy(),color=['Vessel size category'],mask_obs=mask,size=size,na_in_legend=False,frameon=False,save='hurskainen_ven_size.png')
sc.pl.umap(endo_adata.copy(),color=['Mki67'],cmap='viridis',mask_obs=mask,size=size,na_in_legend=False,frameon=False,save='hurskainen_ven_mki67.png')


In [ ]:

os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1,1))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15
for gene in ['Cxcl12','Cxcr4','Ackr3','Esr2','Dkk2','Moxd1']:
    sc.pl.umap(endo_adata,color=[gene],cmap='viridis',size=size,frameon=False,save=f'hurskainen_{gene}.png')
os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))
sc.settings.figdir = figures
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
size=15

In [ ]:
ls = []
for x,y in zip(endo_adata.obs['Cell Subtype'],endo_adata.obs['Vessel size category']):
    if x =='Cap1':
        ls.append('Cap1')
    else:
        if y =='capillary':
            ls.append(f'Cap1')
            continue
        if x =='Arterial EC':
            x = 'PAEC'
        else:
            x ='PVEC'
        ls.append(f'{x} {y[0].upper()}')
endo_adata.obs['ct_s'] = ls


In [ ]:
sc.set_figure_params(dpi_save=300, fontsize=10, figsize=(1.5,1.5))

gene_ls = [
    'Ccdc85a',
    'Glp1r',
    'Kit',
         'Sox4','Nrp1','Ifitm3',
         'Ptprr','Adgrg6','Foxo1',
         'Mgp','Eln','Nr4a2',
][::-1]
sc.pl.dotplot(endo_adata,gene_ls,groupby='ct_s',cmap='viridis',categories_order=['PAEC L','PAEC M','PAEC S','Cap1','PVEC S','PVEC M', 'PVEC L'],
              standard_scale='var',save='size_axis.png')
sc.pl.umap(endo_adata,color=gene_ls,hspace=0.5,wspace=0.3,cmap='viridis',save='size_axis.png')


In [ ]:
gene_ls = ['Dkk2','Sox6',
           'Lama3','Bdkrb2',
           # 'Depp1', not present
           'Stc1',

           'Adam23','Ntrk2',
           'Emid1','Chrm2',
           'Chrm3','Rarb',
           # 'Fads2b', not present
           'Gria3',
           'Ptger3','Moxd1',
             ]
sc.pl.MatrixPlot(endo_adata,gene_ls,groupby='ct_s',categories_order=['PAEC L','PAEC M','PAEC S','Cap1','PVEC S','PVEC M', 'PVEC L'],
              standard_scale='var').style(cmap='viridis').swap_axes().savefig(f'{figures}/matrixplot_size_markers.png',dpi=300,bbox_inches='tight')
sc.pl.DotPlot(endo_adata,gene_ls,groupby='ct_s',categories_order=['PAEC L','PAEC M','PAEC S','Cap1','PVEC S','PVEC M', 'PVEC L'],
              standard_scale='var').style(cmap='viridis').savefig(f'{figures}/matrixplot_size_markers.png',dpi=300,bbox_inches='tight')